In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import astropy.units as u
import astropy.constants as const
from astropy.time import Time, TimeDelta
from astropy.table import Table

import pytz

from astropy.coordinates import SkyCoord, AltAz, EarthLocation, get_moon
from astropy.coordinates import Angle
from astroplan import Observer, FixedTarget, ObservingBlock, is_event_observable, AtNightConstraint, AirmassConstraint

from astroplan import time_grid_from_range

from astroplan.plots import plot_airmass

import astroplan as ap


In [2]:
my_column_name = ["Name", "RAh", "RAm", "RAs", "DEd", "DEm", "DEs", "FName", "Type"]

raw_full_pd = pd.read_csv("galaxytable.txt", skiprows = 24, delimiter = " ", header = None, names = my_column_name)
raw_sample_pd = raw_full_pd.sample(n=10)

raw_sample = Table.from_pandas(raw_sample_pd)

raw_sample_pd_copy = raw_sample_pd.copy()

raw_sample_pd_copy['RAm'] = raw_sample_pd_copy['RAm'] / 60
raw_sample_pd_copy['RAs'] = raw_sample_pd_copy['RAs'] / 3600

raw_sample_pd_copy['DEm'] = raw_sample_pd_copy['DEm'] / 60
raw_sample_pd_copy['DEs'] = raw_sample_pd_copy['DEs'] / 3600


In [3]:
sample = Table.from_pandas(raw_sample_pd_copy)

sample['RAh'] = (sample['RAh'] * u.hourangle).to(u.deg)
sample['RAm'] = (sample['RAm'] * u.hourangle).to(u.deg)
sample['RAs'] = (sample['RAs'] * u.hourangle).to(u.deg)

sample['DEd'] = sample['DEd'] * u.deg
sample['DEm'] = sample['DEm'] * u.deg
sample['DEs'] = sample['DEs'] * u.deg

sample['RA'] = sample['RAh'] + sample['RAm'] + sample['RAs']

sample['Dec'] = sample['DEd'] + sample['DEm'] + sample['DEs']

sample.remove_columns(['RAh', 'RAm', 'RAs', 'DEd', 'DEm', 'DEs', 'Type', 'FName'])

In [4]:
cfht = Observer(latitude = 19.8253 * u.deg,
               longitude = -155.47 * u.deg,
               elevation = 4204 * u.m,
               timezone = 'US/Hawaii',
               name = "CFHT"
              )

utcoffset = 10 * u.hr #HST = UTC -10
sep1 = Time("2019-09-01 12:00:00") 
#objs = SkyCoord(ra = sample['RA'], dec = sample['Dec'], frame = 'icrs')
#sample['RA'][0]
#obj_test = SkyCoord(ra = sample['RA'][0] * u.deg, dec = sample['Dec'][0] * u.deg, frame = 'icrs')

#midnight_cfht = cfht.midnight(sep1, which='next')

#cfht.target_is_up(midnight_cfht, obj_test)

#cfht.altaz(midnight_cfht, obj_test).az

In [5]:
class ObservationWindow(object):
    
    def __init__(self, idx=None, ra=None, dec=None, loc=None, date=None, utcoffset=None):
        self.idx = idx
        self.ra = ra
        self.dec = dec
        self.loc = loc
        self.date = date
        self.utcoffset = utcoffset
        
        
    def obj(self):
        return SkyCoord(ra = self.ra, dec = self.dec, frame = 'icrs')
    
    def obj_rise(self):
        return self.loc.target_rise_time(self.date, self.obj(), which='next') 
    
    def earth_loc(self):
        return self.loc.location
    
    def phase(self):
        return self.loc.moon_phase(self.date)
        
        
    def moon_loc_closest(self):
        if len(self.masked_times()) > 1:
            coords = get_moon(self.masked_times(), location=self.earth_loc())
            separations = coords.separation(self.obj())
            closest = min(separations)
        else:
            closest = float('NaN') * u.deg
        return closest
    
    def moon_sep_transit(self):
        transit = self.loc.target_meridian_transit_time(self.date, self.obj(), which="next")
        moon =  get_moon(transit, location=self.earth_loc())
        return moon.separation(self.obj())
    
    def obj_set(self):
        return self.loc.target_set_time(self.date, self.obj(), which='next')
        
    
    def ast_twilight_set(self):
        return self.loc.twilight_evening_astronomical(self.date, which='next')- self.utcoffset
    
    def ast_twilight_rise(self):
        return self.loc.twilight_morning_astronomical(self.date, which='next')- self.utcoffset
    
    def night(self):
        return np.linspace(0, 12, 1000)*u.hour
    
    def frame_night(self):
        return self.ast_twilight_rise() +self.night()
        
    def up_mask(self):
        constraints = [AtNightConstraint(), AirmassConstraint(1.75)]
        mask = np.transpose(np.where(is_event_observable(constraints, 
                                                        self.loc, self.obj(), 
                                                        times = self.frame_night()
                                                        )
                            )
                   )
        return mask
        
    def masked_times(self):
        return self.frame_night()[self.up_mask()[:,1]]
    
    def visibility_rise(self):
        if len(self.masked_times()) > 1:
            rise = self.masked_times()[0].strftime("%H:%M:%S")
        else:
            rise = 'None'
        return rise
    
    def visibility_set(self):
        if len(self.masked_times()) > 1:
            vset = self.masked_times()[-1].strftime("%H:%M:%S")
        else:
            vset = 'None'
        return vset
    
    def visibility_window(self):
        rise = self.masked_times()[0]
        vset = self.masked_times()[-1]
        print((vset()) - (rise()))
        #return Time(vset()) - Time(rise())
        
        
    def airmass_win(self):
        return self.loc.target_rise_time(self.date, self.obj(), which='next')
        

In [8]:
constraints = [ap.AtNightConstraint(), ap.AirmassConstraint(1.75)]
mask = np.transpose(np.where(ap.is_event_observable(constraints, 
                                                    cfht, test.obj(), 
                                                    times = test.frame_night())
                            )
                   )
masked_times = test.frame_night()[mask[:,1]] 

if len(masked_times) > 1:
    window = [masked_times[0], masked_times[-1]]
else:
    window = False

utcoffset = TimeDelta(36000, format='sec')

#sep1 - masked_times
#for x in masked_times:
    #print(cfht.astropy_time_to_datetime(x))

#np.transpose(ap.is_event_observable(ap.AtNightConstraint(), cfht, test.obj(), times = test.frame_night()))

In [14]:
sample2 = sample.copy()
data_observ = ObservationWindow(ra = sample['RA'], 
                         dec = sample['Dec'], 
                         loc=cfht, 
                         date=sep1+1, 
                         utcoffset=utcoffset)


print("""All times UTC.
Min sep is minimum separation between moon and object in observing window.
Sep transit is separation between moon and object at object\'s meridian transit""")

for day in range(3):
    days = day * 7
    risename = 'Rise 9/{0}'.format(1+days)
    setname = 'Set 9/{0}'.format(1+days)
    closestmoon = 'Min sep 9/{0}'.format(1+days)
    transitmoon = 'Sep transit 9/{0}'.format(1+days)
    phase = 'Moon phase 9/{0}'.format(1+days)
    sample2[risename] = [None] * len(sample2)
    sample2[setname] = [None] * len(sample2)
    sample2[closestmoon] = [None] * len(sample2)
    sample2[transitmoon] = [None] * len(sample2)
    sample2[phase] = [None] * len(sample2)
    
    for x in range(len(sample2)):
        observe = ObservationWindow(ra = sample['RA'][x] * u.deg, 
                             dec = sample['Dec'][x] * u.deg, 
                             loc=cfht, 
                             date=sep1+days*u.day, 
                             utcoffset=utcoffset)
        sample2[risename][x] = observe.visibility_rise()
        sample2[setname][x] = observe.visibility_set()
        sample2[closestmoon][x] = observe.moon_loc_closest().to(u.deg)
        sample2[transitmoon][x] = observe.moon_sep_transit().to(u.deg)
        sample2[phase][x] = observe.phase()
    print(sample2['Name', 'RA', 'Dec', risename, setname, closestmoon, transitmoon, phase])
    print()


All times UTC.
Min sep is minimum separation between moon and object in observing window.
Sep transit is separation between moon and object at object's meridian transit
 Name         RA         ... Sep transit 9/1     Moon phase 9/1   
             deg         ...                                      
----- ------------------ ... --------------- ---------------------
13627 351.77374999999995 ... 146d25m23.0894s 2.621778944464933 rad
 9787 249.16249999999997 ...  63d50m52.0754s 2.621778944464933 rad
12620  329.0487499999999 ... 128d13m22.5078s 2.621778944464933 rad
 7087 191.99291666666664 ...   4d42m17.1648s 2.621778944464933 rad
  902             11.705 ... 177d46m00.8292s 2.621778944464933 rad
 4552  49.47249999999999 ... 139d17m40.2988s 2.621778944464933 rad
  807 11.256666666666664 ... 173d34m07.9203s 2.621778944464933 rad
13773  352.1916666666666 ... 146d31m17.5224s 2.621778944464933 rad
 3411  25.92833333333333 ... 164d13m24.1074s 2.621778944464933 rad
 8443 228.97999999999996 ..

In [10]:
oct1 = Time("2019-10-01 12:00:00") 

sample3 = sample.copy()
for day in range(3):
    days = day * 7
    risename = 'Rise 10/{0}'.format(1+days)
    setname = 'Set 10/{0}'.format(1+days)
    closestmoon = 'Min sep 10/{0}'.format(1+days)
    transitmoon = 'Sep transit 10/{0}'.format(1+days)
    phase = 'Moon phase 10/{0}'.format(1+days)
    sample3[risename] = [None] * len(sample3)
    sample3[setname] = [None] * len(sample3)
    sample3[closestmoon] = [None] * len(sample3)
    sample3[transitmoon] = [None] * len(sample3)
    sample3[phase] = [None] * len(sample3)
    
    for x in range(len(sample3)):
        observe2 = ObservationWindow(ra = sample['RA'][x] * u.deg, 
                             dec = sample['Dec'][x] * u.deg, 
                             loc=cfht, 
                             date=oct1+days*u.day, 
                             utcoffset=utcoffset)
        sample3[risename][x] = observe2.visibility_rise()
        sample3[setname][x] = observe2.visibility_set()
        sample3[closestmoon][x] = observe2.moon_loc_closest().to(u.deg)
        sample3[transitmoon][x] = observe2.moon_sep_transit().to(u.deg)
        sample3[phase][x] = observe2.phase()
    print(sample3['Name', 'RA', 'Dec', risename, setname, closestmoon, transitmoon, phase])
    print()


 Name         RA         ... Sep transit 10/1    Moon phase 10/1    
             deg         ...                                        
----- ------------------ ... ---------------- ----------------------
13627 351.77374999999995 ...  110d53m10.6493s 2.4734029915585083 rad
 9787 249.16249999999997 ...   58d35m57.2253s 2.4734029915585083 rad
12620  329.0487499999999 ...   94d01m33.8869s 2.4734029915585083 rad
 7087 191.99291666666664 ...    40d18m32.676s 2.4734029915585083 rad
  902             11.705 ...  132d00m33.2503s 2.4734029915585083 rad
 4552  49.47249999999999 ...  150d24m20.3549s 2.4734029915585083 rad
  807 11.256666666666664 ...  133d54m03.3476s 2.4734029915585083 rad
13773  352.1916666666666 ...  111d01m31.6524s 2.4734029915585083 rad
 3411  25.92833333333333 ...   145d01m37.929s 2.4734029915585083 rad
 8443 228.97999999999996 ...   23d04m56.7289s 2.4734029915585083 rad

 Name         RA         ... Sep transit 10/8    Moon phase 10/8    
             deg         ...     

In [15]:

sample2

Name,RA,Dec,Rise 9/1,Set 9/1,Min sep 9/1,Sep transit 9/1,Moon phase 9/1,Rise 9/8,Set 9/8,Min sep 9/8,Sep transit 9/8,Moon phase 9/8,Rise 9/15,Set 9/15,Min sep 9/15,Sep transit 9/15,Moon phase 9/15
,deg,deg,,,,,,,,,,,,,,,
int64,float64,float64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
13627,351.77374999999995,-10.995277777777778,08:03:17,14:13:44,158d22m42.7638s,146d25m23.0894s,2.621778944464933 rad,07:35:57,13:46:24,65d21m44.2856s,54d42m45.7789s,1.087491643021969 rad,07:08:21,13:18:48,16d24m00.2614s,29d28m49.032s,0.2636694654159036 rad
9787,249.16249999999997,40.815916666666666,04:51:35,08:20:35,68d24m40.0074s,63d50m52.0754s,2.621778944464933 rad,04:53:48,07:52:32,70d11m06.7936s,75d40m23.3206s,1.087491643021969 rad,04:55:44,07:24:56,112d06m42.8469s,116d15m18.4065s,0.2636694654159036 rad
12620,329.0487499999999,0.27408333333333335,06:07:58,13:08:09,139d00m50.1045s,128d13m22.5078s,2.621778944464933 rad,05:40:38,12:40:49,50d02m31.8245s,40d15m55.6732s,1.087491643021969 rad,05:13:02,12:13:13,36d43m39.5042s,48d56m06.8615s,0.2636694654159036 rad
7087,191.99291666666664,-3.468027777777778,None,None,nan deg,4d42m17.1648s,2.621778944464933 rad,None,None,nan deg,97d23m36.371s,1.087491643021969 rad,None,None,nan deg,176d14m01.4342s,0.2636694654159036 rad
902,11.705,-1.2581944444444444,09:00:57,15:55:22,175d00m13.0227s,177d46m00.8292s,2.621778944464933 rad,08:33:37,15:28:02,86d16m06.9841s,76d09m31.7122s,1.087491643021969 rad,08:06:01,15:00:25,3d12m46.357s,7d50m18.1174s,0.2636694654159036 rad
4552,49.47249999999999,-16.476305555555555,12:10:30,16:09:46,137d31m42.0009s,139d17m40.2988s,2.621778944464933 rad,11:42:27,16:11:16,112d30m52.0036s,113d25m03.2305s,1.087491643021969 rad,11:15:33,16:12:30,42d34m00.8356s,43d04m45.3352s,0.2636694654159036 rad
807,11.256666666666664,5.094694444444444,08:48:42,16:04:44,171d33m44.8522s,173d34m07.9203s,2.621778944464933 rad,08:20:38,15:36:41,88d12m36.2342s,78d17m12.393s,1.087491643021969 rad,07:53:02,15:09:04,7d20m27.7865s,8d00m28.5696s,0.2636694654159036 rad
13773,352.1916666666666,-11.566138888888888,08:06:53,14:13:44,158d28m12.0451s,146d31m17.5224s,2.621778944464933 rad,07:38:50,13:46:24,65d33m08.9609s,54d53m45.6729s,1.087491643021969 rad,07:11:57,13:18:48,16d19m57.592s,29d21m36.616s,0.2636694654159036 rad


In [13]:
sample3

Name,RA,Dec,Rise 10/1,Set 10/1,Min sep 10/1,Sep transit 10/1,Moon phase 10/1,Rise 10/8,Set 10/8,Min sep 10/8,Sep transit 10/8,Moon phase 10/8,Rise 10/15,Set 10/15,Min sep 10/15,Sep transit 10/15,Moon phase 10/15
,deg,deg,,,,,,,,,,,,,,,
int64,float64,float64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
13627,351.77374999999995,-10.995277777777778,06:05:15,12:15:42,122d29m45.5712s,110d53m10.6493s,2.4734029915585083 rad,05:38:06,11:48:33,33d22m20.3368s,22d37m56.7153s,1.0245464793959056 rad,05:10:20,11:20:47,48d36m28.4078s,62d27m10.8933s,0.3254627748787247 rad
9787,249.16249999999997,40.815916666666666,04:59:40,06:22:33,58d53m46.921s,58d35m57.2253s,2.4734029915585083 rad,05:01:21,05:54:41,86d44m02.3073s,92d27m06.6093s,1.0245464793959056 rad,05:03:07,05:27:38,121d42m54.7022s,122d31m46.2486s,0.3254627748787247 rad
12620,329.0487499999999,0.27408333333333335,04:59:40,11:10:07,105d02m59.325s,94d01m33.8869s,2.4734029915585083 rad,05:01:21,10:42:58,22d29m11.3367s,17d05m37.9834s,1.0245464793959056 rad,05:03:07,10:15:12,67d27m28.6773s,80d05m35.9492s,0.3254627748787247 rad
7087,191.99291666666664,-3.468027777777778,None,None,nan deg,40d18m32.676s,2.4734029915585083 rad,None,None,nan deg,129d00m26.3236s,1.0245464793959056 rad,None,None,nan deg,148d10m16.426s,0.3254627748787247 rad
902,11.705,-1.2581944444444444,07:02:54,13:57:19,143d09m35.1869s,132d00m33.2503s,2.4734029915585083 rad,06:35:02,13:30:10,54d26m58.5549s,44d08m39.7782s,1.0245464793959056 rad,06:07:59,13:02:24,27d03m44.2716s,40d56m58.0631s,0.3254627748787247 rad
4552,49.47249999999999,-16.476305555555555,10:12:27,15:49:02,150d02m48.4149s,150d24m20.3549s,2.4734029915585083 rad,09:44:35,15:21:53,84d11m36.9147s,85d34m06.5985s,1.0245464793959056 rad,09:17:32,14:54:07,28d44m22.2109s,28d56m38.9765s,0.3254627748787247 rad
807,11.256666666666664,5.094694444444444,06:50:39,14:06:41,144d45m14.0165s,133d54m03.3476s,2.4734029915585083 rad,06:22:47,13:38:49,56d34m57.8226s,46d28m44.4226s,1.0245464793959056 rad,05:55:01,13:11:46,25d31m38.1275s,39d12m33.0953s,0.3254627748787247 rad
13773,352.1916666666666,-11.566138888888888,06:08:51,12:16:25,122d38m20.2324s,111d01m31.6524s,2.4734029915585083 rad,05:40:59,11:48:33,33d35m26.0418s,22d51m35.5663s,1.0245464793959056 rad,05:13:56,11:20:47,48d29m02.9532s,62d18m08.1132s,0.3254627748787247 rad
